In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # Loads .env file into environment
token = os.getenv("GITHUB_TOKEN")
headers = {"Authorization": f"token {token}"}


In [5]:
import os
import subprocess
import requests
import json
import time

with open("all_repos.json", "r") as f:
    final_repos = json.load(f)

# Directory to save data
output_dir = "downloaded_repos"
os.makedirs(output_dir, exist_ok=True)

def get_github_api(url, params=None):
    r = requests.get(url, headers=headers, params=params)
    if r.status_code == 200:
        return r.json()
    else:
        print(f"Failed to fetch {url}: {r.status_code}")
        return None

repo_details = {}

i= 0
for repo_full_name in final_repos[::-1]:
    print(f"[{i} / {len(final_repos)}] Processing {repo_full_name}...")
    i += 1
    repo_data = {}

    # 1. Download files (list of files in the repo)
    # Also download the content of each file (if it's not a directory)
    # Clone the repository into a subdirectory if not already cloned
    repo_dir = os.path.join(output_dir, repo_full_name.replace('/', '__'))
    if not os.path.exists(repo_dir):
        clone_url = f"https://github.com/{repo_full_name}.git"
        try:
            subprocess.run(
                ["git", "clone", "--depth", "500", clone_url, repo_dir],
                check=True,
                timeout=10
            )
        except Exception as e:
            print(f"  Failed to clone {repo_full_name}: {e}")

    # List the project tree
    project_files = []
    for root, dirs, files in os.walk(repo_dir):
        for file in files:
            rel_path = os.path.relpath(os.path.join(root, file), repo_dir)
            project_files.append(rel_path)

    repo_data['files'] = project_files

    # Add file contents for text-like files
    file_contents = {}
    for rel_path in project_files:
        abs_path = os.path.join(repo_dir, rel_path)
        try:
            # Read only small files (e.g., <1MB) and skip binaries
            if os.path.getsize(abs_path) < 1024 * 1024:
                with open(abs_path, "rb") as f:
                    raw = f.read()
                    try:
                        text = raw.decode("utf-8")
                    except UnicodeDecodeError:
                        try:
                            text = raw.decode("latin1")
                        except Exception:
                            continue  # skip non-text files
                    # Heuristic: skip if too many non-printable characters
                    if sum(c < " " and c not in "\n\r\t" for c in text) < 0.05 * len(text):
                        file_contents[rel_path] = text
        except Exception:
            continue  # skip unreadable files

    repo_data['file_contents'] = file_contents


    print(f"  Found {len(repo_data['files'])} files in {repo_full_name}")

    # 2. Contributors
    contributors_url = f"https://api.github.com/repos/{repo_full_name}/contributors"
    contributors = get_github_api(contributors_url)
    repo_data['contributors'] = contributors if contributors is not None else []

    print(f"  Found {len(repo_data['contributors'])} contributors in {repo_full_name}")

    # 3. Commits with statistics
    # Get commit statistics from the local git repository
    commits = []
    try:
        # Get the list of commit hashes (limit to last 100 for performance)
        result = subprocess.run(
            ["git", "-C", repo_dir, "rev-list", "--max-count=500", "HEAD"],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True
        )
        commit_hashes = result.stdout.strip().split('\n')
        for sha in commit_hashes:
            # Get commit info and stats
            show_cmd = [
                "git", "-C", repo_dir, "show", "--quiet", "--format=%H|%an|%ae|%ad|%s", sha
            ]
            stat_cmd = [
                "git", "-C", repo_dir, "show", "--stat", "--oneline", "--format=", sha
            ]
            show_result = subprocess.run(show_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            stat_result = subprocess.run(stat_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
            if show_result.returncode == 0:
                parts = show_result.stdout.strip().split('|')
                if len(parts) == 5:
                    commit_info = {
                        "sha": parts[0],
                        "author_name": parts[1],
                        "author_email": parts[2],
                        "date": parts[3],
                        "message": parts[4],
                        "stats": stat_result.stdout.strip()
                    }
                    commits.append(commit_info)
    except Exception as e:
        print(f"  Failed to get commits for {repo_full_name}: {e}")

    repo_data['commits'] = commits

    print(f"  Found {len(repo_data['commits'])} commits in {repo_full_name}")

    # 4. Issues
    issues_url = f"https://api.github.com/repos/{repo_full_name}/issues"
    issues = []
    params = {"state": "all", "per_page": 100, "page": 1}
    while True:
        issues_page = get_github_api(issues_url, params=params)
        if not issues_page:
            break
        issues.extend(issues_page)
        
        break
    repo_data['issues'] = issues

    print(f"  Found {len(repo_data['issues'])} issues in {repo_full_name}")

    # Save per repo
    with open(os.path.join(output_dir, f"{repo_full_name.replace('/', '__')}.json"), "w") as f:
        json.dump(repo_data, f, indent=2)

    repo_details[repo_full_name] = repo_data

    # To avoid hitting rate limits
    time.sleep(0.2)

# Save all details to a single file
with open("all_repo_details.json", "w") as f:
    json.dump(repo_details, f, indent=2)

print(f"Saved details for {len(final_repos)} repositories in {output_dir}")

[0 / 1911] Processing asciiman/react-native-native-listview...
  Found 63 files in asciiman/react-native-native-listview
  Found 1 contributors in asciiman/react-native-native-listview
  Found 18 commits in asciiman/react-native-native-listview
  Found 15 issues in asciiman/react-native-native-listview
[1 / 1911] Processing linux-china/toolchains-maven-plugin...
  Found 46 files in linux-china/toolchains-maven-plugin
  Found 2 contributors in linux-china/toolchains-maven-plugin
  Found 72 commits in linux-china/toolchains-maven-plugin
  Found 13 issues in linux-china/toolchains-maven-plugin
[2 / 1911] Processing lrwinx/shop...
  Found 99 files in lrwinx/shop
  Found 2 contributors in lrwinx/shop
  Found 43 commits in lrwinx/shop
  Found 2 issues in lrwinx/shop
[3 / 1911] Processing Jaouan/ViewsFrom...


Cloning into 'downloaded_repos/Jaouan__ViewsFrom'...


  Found 86 files in Jaouan/ViewsFrom
  Found 1 contributors in Jaouan/ViewsFrom
  Found 7 commits in Jaouan/ViewsFrom
  Found 1 issues in Jaouan/ViewsFrom
[4 / 1911] Processing bufferapp/BufferTextInputLayout...


Cloning into 'downloaded_repos/bufferapp__BufferTextInputLayout'...


  Found 81 files in bufferapp/BufferTextInputLayout
  Found 4 contributors in bufferapp/BufferTextInputLayout
  Found 23 commits in bufferapp/BufferTextInputLayout
  Found 9 issues in bufferapp/BufferTextInputLayout
[5 / 1911] Processing vanga999/nSocket...


...

  Found 73 files in electronicarts/ea-agent-loader
  Found 3 contributors in electronicarts/ea-agent-loader
  Found 84 commits in electronicarts/ea-agent-loader
  Found 10 issues in electronicarts/ea-agent-loader
[1909 / 1911] Processing ferstl/pedantic-pom-enforcers...


Cloning into 'downloaded_repos/ferstl__pedantic-pom-enforcers'...


  Found 136 files in ferstl/pedantic-pom-enforcers
  Found 4 contributors in ferstl/pedantic-pom-enforcers
  Found 496 commits in ferstl/pedantic-pom-enforcers
  Found 48 issues in ferstl/pedantic-pom-enforcers
[1910 / 1911] Processing mojohaus/versions...


Cloning into 'downloaded_repos/mojohaus__versions'...


  Found 1362 files in mojohaus/versions
  Found 30 contributors in mojohaus/versions
  Found 498 commits in mojohaus/versions
  Found 100 issues in mojohaus/versions
Saved details for 1911 repositories in downloaded_repos
